# SQLAlchemy
This notebook shows an example of how to use SQLAlchemy module in python to perform CRUD operations in a SQL database (PostgreSQL in this case). Also you will see the link with pandas package.

## 1. Libraries importation

In [1]:
# For DB connection and queries handling
from sqlalchemy import create_engine, text

# For credentials
from configparser import ConfigParser

# Pandas
import pandas as pd

# Sys for exit needed
import sys

# For paths
import os

## 2. Create the engine (DB connection)
SQLAlchemy gives us a few options for managing database connections, but they all begin with the concept of an engine. An "engine" is a Python object representing a database. The syntax for creating an engine is refreshingly simple. The method `create_engine()` requires one positional argument, which is a URI representing the connection information to connect to a database. This URI has the following structure:

`[DB_TYPE]+[DB_CONNECTOR]://[USERNAME]:[PASSWORD]@[HOST]:[PORT]/[DB_NAME]`

Where:
- `DB_TYPE`: is the DB type, such as MySQL (mysql), PostgreSQL (postgresql), SQLite (sqlite), Oracle (oracle), etc.
- `DB_CONNECTOR`: is the database connection library (dialect) you chose to use. For PostgreSQL: psycopg2, pg8000.
- `USERNAME` & `PASSWORD`: the user and password to access the DB.
- `HOST` & `PORT`: the DB URL and port.
- `DB_NAME`: the name of the DB you want to access.

In [2]:
# Parse data from the .ini file
config = ConfigParser()

# Path to the .ini file
ini_path = os.path.join(os.getcwd(), 'db_data.ini')

# Read the configuration file
if os.path.isfile(ini_path):
    config.read(ini_path)
else:
    print('ERROR: Please, check the db_data.ini file.')
    sys.exit()

# Load the credentials
USER = config['PostgreSQL']['user']
PASS = config['PostgreSQL']['pass']
HOST = config['PostgreSQL']['host']
PORT = config['PostgreSQL']['port']
DB_NAME = config['PostgreSQL']['db_name']

try: 
    # Connect to the DB
    engine = create_engine(
        f"postgresql+psycopg2://{USER}:{PASS}@{HOST}:{PORT}/{DB_NAME}"
    )
except:
    print('Something went wrong with the DB connection')
    sys.exit()

## 3. CRUD Operations
Once the connection is stablished, there are 2 ways to perform CRUD operations:
- Directly with the `engine` object (called explicit connectionless execution), meaning no `commit()` is needed.
- With a [scopped session](https://docs.sqlalchemy.org/en/14/orm/contextual.html#unitofwork-contextual), which needs the `commit()` execution after the query statement.

In this notebook we'll use conectionless execution, but in the practice it's a very little difference between the two methods.

The `text()` method escapes dangerous characters found in queries in order to avoid problems with the execution.

### 3.1 Create
Fisrt, let's create the tables we are going to work with.

In [3]:
# Define the query
q = """
    CREATE TABLE IF NOT EXISTS aerolineas
    (
        id_aerolinea INT NOT NULL,
        nombre_aerolinea VARCHAR(20),
        PRIMARY KEY(id_aerolinea)
    );
    """

# Execute it
engine.execute(text(q))

# Define the query
q = """
    CREATE TABLE IF NOT EXISTS aeropuertos
    (
        id_aeropuerto INT NOT NULL,
        nombre_aeropuerto VARCHAR(20),
        PRIMARY KEY(id_aeropuerto)
    );
    """

# Execute it
engine.execute(text(q))

# Define the query
q = """
    CREATE TABLE IF NOT EXISTS movimientos
    (
        id_movimiento INT NOT NULL,
        descripcion VARCHAR(20),
        PRIMARY KEY(id_movimiento)
    );
    """

# Execute it
engine.execute(text(q))

# Define the query
q = """
    CREATE TABLE vuelos
    (
        id_aerolinea INT NOT NULL,
        id_aeropuerto INT NOT NULL,
        id_movimiento INT NOT NULL,
        dia DATE
    );
    """

# Execute it
engine.execute(text(q))

Now, let's insert some rows in each table.

In [4]:
engine.execute(
    text(
        """
        INSERT INTO aerolineas VALUES
        (1, 'Volaris'), (2, 'Aeromar'), 
        (3, 'Interjet'), (4, 'Aeromexico');
        """
    )
)

engine.execute(
    text(
        """
        INSERT INTO aeropuertos VALUES
        (1, 'Benito Juarez'), (2, 'Guanajuato'), 
        (3, 'La Paz'), (4, 'Oaxaca');
        """
    )
)

engine.execute(
    text(
        """
        INSERT INTO movimientos VALUES
        (1, 'Salida'), (2, 'Llegada');
        """
    )
)

engine.execute(
    text(
        """
        INSERT INTO vuelos VALUES
        (1, 1, 1, '2021-05-02'), 
        (2, 1, 1, '2021-05-02'), 
        (3, 2, 2, '2021-05-02'), 
        (4, 3, 2, '2021-05-02'), 
        (1, 3, 2, '2021-05-02'), 
        (2, 1, 1, '2021-05-02'), 
        (2, 3, 1, '2021-05-04'), 
        (3, 4, 1, '2021-05-04'), 
        (3, 4, 1, '2021-05-04');
        """
    )
)

### 3.2 Read
Let's visualize the entire tables and then let's ask some questions.

In [5]:
# Table aerolíneas
result = engine.execute(text("select * FROM aerolineas;"))
print('Table: Aerolíneas')
print(f'There were {result.rowcount} rows selected.')
print(tuple(result.keys()))
for row in result.fetchall():
    print(row)

# Table aeropuertos
result = engine.execute(text("SELECT * FROM aeropuertos;"))
print('\nTable: Aeropuertos')
print(f'There were {result.rowcount} rows selected.')
print(tuple(result.keys()))
for row in result.fetchall():
    print(row)


# Table movimientos
result = engine.execute(text("SELECT * FROM movimientos;"))
print('\nTable: Movimientos')
print(f'There were {result.rowcount} rows selected.')
print(tuple(result.keys()))
for row in result.fetchall():
    print(row)


# Table vuelos
result = engine.execute(
    text(
        """
        SELECT 
        id_aerolinea, 
        id_aeropuerto,
        id_movimiento,
        TO_CHAR(dia, 'DD/MM/YYYY') AS dia
        FROM vuelos;
        """
    )
)
print('\nTable: Vuelos')
print(f'There were {result.rowcount} rows selected.')
print(tuple(result.keys()))
for row in result.fetchall():
    print(row)


Table: Aerolíneas
There were 4 rows selected.
('id_aerolinea', 'nombre_aerolinea')
(1, 'Volaris')
(2, 'Aeromar')
(3, 'Interjet')
(4, 'Aeromexico')

Table: Aeropuertos
There were 4 rows selected.
('id_aeropuerto', 'nombre_aeropuerto')
(1, 'Benito Juarez')
(2, 'Guanajuato')
(3, 'La Paz')
(4, 'Oaxaca')

Table: Movimientos
There were 2 rows selected.
('id_movimiento', 'descripcion')
(1, 'Salida')
(2, 'Llegada')

Table: Vuelos
There were 9 rows selected.
('id_aerolinea', 'id_aeropuerto', 'id_movimiento', 'dia')
(1, 1, 1, '02/05/2021')
(2, 1, 1, '02/05/2021')
(3, 2, 2, '02/05/2021')
(4, 3, 2, '02/05/2021')
(1, 3, 2, '02/05/2021')
(2, 1, 1, '02/05/2021')
(2, 3, 1, '04/05/2021')
(3, 4, 1, '04/05/2021')
(3, 4, 1, '04/05/2021')


¿What is the name of the airport that has had the most movement during the year?

In [6]:
result = engine.execute(
    text(
        """
        SELECT nombre_aeropuerto
        FROM aeropuertos
        INNER JOIN vuelos
        ON vuelos.id_aeropuerto = aeropuertos.id_aeropuerto
        GROUP BY nombre_aeropuerto
        HAVING COUNT(vuelos.id_movimiento) = 
        (
            SELECT COUNT(id_movimiento) AS mov_count
            FROM vuelos
            GROUP BY id_aeropuerto
            ORDER BY mov_count DESC 
            LIMIT 1
        );
        """
    )
)

print(tuple(result.keys())[0])
for row in result.fetchall():
    print(row[0])

nombre_aeropuerto
Benito Juarez
La Paz


¿What is the airline name you have made greatest number of flights during the year?

In [7]:
result = engine.execute(
    text(
        """
        SELECT nombre_aerolinea
        FROM aerolineas
        INNER JOIN vuelos
        ON vuelos.id_aerolinea = aerolineas.id_aerolinea
        GROUP BY nombre_aerolinea
        HAVING COUNT(vuelos.id_movimiento) = 
        (
            SELECT COUNT(id_movimiento) AS mov_count
            FROM vuelos
            GROUP BY id_aerolinea
            ORDER BY mov_count DESC 
            LIMIT 1
        );
        """
    )
)

print(tuple(result.keys())[0])
for row in result.fetchall():
    print(row[0])

nombre_aerolinea
Aeromar
Interjet


¿On which day have there been the greatest number of flights?

In [8]:
result = engine.execute(
    text(
        """
        SELECT 
        TO_CHAR(dia, 'DD/MM/YYYY') AS dia
        FROM vuelos
        GROUP BY dia
        HAVING COUNT(dia) = 
        (
            SELECT COUNT(dia)
            FROM vuelos 
            GROUP BY dia
            ORDER BY dia ASC
            LIMIT 1
        );
        """
    )
)

print(tuple(result.keys())[0])
for row in result.fetchall():
    print(row[0])

dia
02/05/2021


### 3.3 Update